In [21]:
import tensorflow as tf
from tensorflow.keras.models import load_model
import pickle
import pandas as pd
import numpy as np
import os


In [22]:
## Import the trained model
model = load_model('model.h5')
## Load the geography encoder
with open('one_hot_encoder_geography.pkl', 'rb') as f:
    geography_encoder = pickle.load(f)
## Load the gender encoder
with open('label_encoder_gender.pkl', 'rb') as f:
    gender_encoder = pickle.load(f)

## Load the scaler
with open('scaler.pkl', 'rb') as f:
    scaler = pickle.load(f)

In [23]:
# Example input data
input_data = {
    'CreditScore': 600,
    'Geography': 'Spain',
    'Gender': 'Female',
    'Age': 55,
    'Tenure': 1,
    'Balance': 195660.80,
    'NumOfProducts': 1,
    'HasCrCard': 0,
    'IsActiveMember': 1,
    'EstimatedSalary': 1125442.58
}


# input_data = {
#     'CreditScore': 500,
#     'Geography': 'Spain',
#     'Gender': 'Female',
#     'Age': 55,
#     'Tenure': 1,
#     'Balance': 19660.80,
#     'NumOfProducts': 1,
#     'HasCrCard': 0,
#     'IsActiveMember': 0,
#     'EstimatedSalary': 112542.58
# }

In [24]:
## One hot encode the geography
geography_encoded = geography_encoder.transform([[input_data['Geography']]])
geography_encoded_df = pd.DataFrame(
	geography_encoded.toarray(), 
	columns=geography_encoder.get_feature_names_out(['Geography'])
)
geography_encoded_df

c:\Users\Suraj Khodade\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


,Geography_France,Geography_Germany,Geography_Spain
0,0.0,0.0,1.0


In [25]:
## Combine the input data with the encoded geography
input_df = pd.DataFrame([input_data])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,Spain,Female,55,1,195660.8,1,0,1,1125442.58


In [26]:
input_df = pd.concat([input_df, geography_encoded_df], axis=1)
input_df = input_df.drop(columns=['Geography'])
input_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,Female,55,1,195660.8,1,0,1,1125442.58,0.0,0.0,1.0


In [27]:
## Encode categorical variables
input_df['Gender'] = gender_encoder.transform(input_df['Gender'])
input_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,0,55,1,195660.8,1,0,1,1125442.58,0.0,0.0,1.0


In [28]:
## Scale the numerical features
input_df_scaled = scaler.transform(input_df)
input_df_scaled 

array([[-0.52544045, -1.10191942,  1.52508472, -1.38666414,  1.91465676,
        -0.91025649, -1.55996038,  0.97067965, 17.89088334, -0.99825153,
        -0.57831252,  1.73089688]])

In [29]:
## predict the churn probability
churn_probability = model.predict(input_df_scaled)
print("Churn Probability:", churn_probability)
print(f"Churn Probability: {churn_probability[0][0]:.2f}")
print("🚀 Prediction completed successfully!")

1/1 [==============================] - 0s 100ms/step
Churn Probability: [[0.31429213]]
Churn Probability: 0.31
🚀 Prediction completed successfully!


In [30]:
pred_probability = churn_probability[0][0]
print(f"Predicted Probability of Churn: {pred_probability:.2f}")
if pred_probability > 0.5:
    print("The customer is likely to churn.")
else:
    print("The customer is likely to stay.")

Predicted Probability of Churn: 0.31
The customer is likely to stay.
